In [ ]:

prompt = """
In this environment you have access to a set of tools you can use to answer the user's question.
You can invoke functions by writing a "<antml:function_calls>" block like the following as part of your reply to the user:
<antml:function_calls>
<antml:invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</antml:parameter>
...
</antml:invoke>
<antml:invoke name="$FUNCTION_NAME2">
...
</antml:invoke>
</antml:function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format.

Here are the functions available in JSONSchema format:
<functions>
<function>{"description": "Finds the tech document based on the layer.\n\n    Args:\n        layer: the layer of tech document to find only include "frontend", "backend"", "name": "find_document", "parameters": {"properties": {"layer": {"type": "string"}}</function>
</functions>

{{ Please identify the nosql databases from the backend doc }}

Answer the user's request using the relevant tool(s), if they are available. Check that all the required parameters for each tool call are provided or can reasonably be inferred from context. IF there are no relevant tools or there are missing values for required parameters, ask the user to supply these values; otherwise proceed with the tool calls. If the user provides a specific value for a parameter (for example provided in quotes), make sure to use that value EXACTLY. DO NOT make up values for or ask about optional parameters. Carefully analyze descriptive terms in the request as they may indicate required parameter values that should be included even if not explicitly quoted.
"""

import dotenv
dotenv.load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",thinking_budget=0)
llm.invoke(prompt).content

'<antml:function_calls>\n<antml:invoke name="find_document">\n<antml:parameter name="layer">backend</antml:parameter>\n</antml:invoke>\n</antml:function_calls>'

In [3]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
    "reference": "The company reported an 8% growth in Q3 2024, primarily driven by strong sales in the Asian market, attributed to strategic marketing and localized products, with continued growth anticipated in the next quarter."
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.13718598426177148

In [47]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
import dotenv
dotenv.load_dotenv()
from ragas.llms import LangchainLLMWrapper
evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"))


/Users/yuliangli/projects/llmops/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
}

metric = AspectCritic(name="summary_accuracy",llm=evaluator_llm, definition="Verify if the summary is accurate.")
test_data = SingleTurnSample(**test_data)
await metric.single_turn_ascore(test_data)

1

In [ ]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness
)
test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
}

metric = AspectCritic(name="summary_accuracy",llm=evaluator_llm, definition="Verify if the summary is accurate.")
test_data = SingleTurnSample(**test_data)
await metric.single_turn_ascore(test_data)

In [2]:
import nest_asyncio
import asyncio
# 启用嵌套异步支持
nest_asyncio.apply()

In [43]:
import dotenv
dotenv.load_dotenv()
from agent_tech_QA_MCP import graph
from langchain_core.messages import HumanMessage,ToolMessage
import uuid

questions = [
    "后端文档中有哪些数据库相关的内容？",
    "前端文档中，有哪些框架可以选择？",
    "后端文档中的数据库，在AWS上部署有什么最佳实践？",
    "Azure上有哪些功能，可以支持前端文档中的最佳实践？"
]
synth = {"questions":questions, "responses":[], "tool_trajectory":[]}
for q in questions:
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    # Start conversation
    input_message = HumanMessage(content=q)
    output = await graph.ainvoke({"messages": [input_message]}, config) 
    synth["responses"].append(output["messages"][-1].content)
    synth["tool_trajectory"].append("[TOOLSEP]".join([m.name for m in output["messages"] if type(m)==ToolMessage]))



In [44]:
import pandas as pd 
pd.DataFrame.from_dict(synth).to_csv("tests/data/agent_tech_QA_MCP_data.csv")

In [45]:
import dotenv
import pandas as pd 
synth_data = pd.read_csv("tests/data/agent_tech_QA_MCP_data.csv")
dotenv.load_dotenv()
from agent_tech_QA_MCP import graph
from langchain_core.messages import HumanMessage,ToolMessage
import uuid
questions = synth_data["questions"].tolist()

outputs = {"questions":questions, "responses":[], "tool_trajectory":[]}
for q in questions:
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    input_message = HumanMessage(content=q)
    output = await graph.ainvoke({"messages": [input_message]}, config) 
    outputs["responses"].append(output["messages"][-1].content)
    outputs["tool_trajectory"].append("[TOOLSEP]".join([m.name for m in output["messages"] if type(m)==ToolMessage]))



{'questions': ['后端文档中有哪些数据库相关的内容？',
  '前端文档中，有哪些框架可以选择？',
  '后端文档中的数据库，在AWS上部署有什么最佳实践？',
  'Azure上有哪些功能，可以支持前端文档中的最佳实践？'],
 'responses': ['后端文档中数据库相关的内容主要包括：\n\n**数据库设计**：\n1.  **关系型数据库**：\n    *   规范化设计\n    *   索引优化\n    *   事务管理\n2.  **NoSQL数据库**：\n    *   文档数据库：MongoDB\n    *   键值数据库：Redis\n    *   图数据库：Neo4j\n\n**性能优化**中也提到了**数据库优化**：\n*   查询优化\n*   连接池\n*   读写分离',
  '根据前端开发最佳实践文档，前端框架有以下选择：\n\n*   **React**: 适合大型应用，生态系统丰富。\n*   **Vue**: 学习曲线平缓，适合中小型项目。\n*   **Angular**: 企业级框架，提供完整的解决方案。',
  '在AWS上部署数据库时，可以参考以下最佳实践：\n\n1.  **Amazon RDS 蓝/绿部署最佳实践**: 如果您使用Amazon RDS，可以参考蓝/绿部署的最佳实践，以实现数据库更新的最小停机时间。\n2.  **AWS Database Migration Service 最佳实践**: 如果您需要进行数据库迁移，AWS Database Migration Service 提供了一系列推荐的最佳实践。\n3.  **迁移大型 MySQL 和 MariaDB 数据库的最佳实践**: 对于大型MySQL和MariaDB数据库迁移到AWS云，AWS Prescriptive Guidance提供了专门的最佳实践。\n4.  **Amazon RDS 安全最佳实践**: 数据库安全至关重要，Amazon RDS提供了安全最佳实践，包括使用AWS Identity and Access Management (IAM) 控制对RDS资源和操作的访问。\n5.  **在AWS上运行Oracle数据库的最佳实践**: 如果您使用Oracle数据库，可以参考AWS提供的白皮书，了解